In [26]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [27]:
loaders = [
    # 파일을 로드합니다.
    TextLoader("./data/ai-story.txt"),
    # 파일을 로드합니다.
    TextLoader("./data/appendix-keywords.txt"),
]

docs = []
for loader in loaders:
    # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
    docs.extend(
        loader.load()
    )
docs

[Document(page_content='Scikit Learn\n\nScikit-learn은 Python 언어를 위한 또 다른 핵심 라이브러리로, 기계 학습의 다양한 알고리즘을 구현하기 위해 설계되었습니다. 이 라이브러리는 2007년 David Cournapeau에 의해 프로젝트가 시작되었으며, 그 후로 커뮤니티의 광범위한 기여를 받아 현재까지 발전해왔습니다. Scikit-learn은 분류, 회귀, 군집화, 차원 축소 등 다양한 기계 학습 작업을 지원하며, 사용하기 쉬운 API와 함께 제공되어 연구자와 개발자가 복잡한 데이터 과학 문제를 해결할 수 있도록 돕습니다.\n\n핵심 특징 중 하나는 다양한 기계 학습 모델을 일관된 인터페이스로 제공한다는 점입니다. 이는 사용자가 알고리즘 간에 쉽게 전환할 수 있게 하여, 최적의 모델을 찾는 과정을 단순화합니다. 또한, Scikit-learn은 사전 처리, 모델 선택, 평가 지표 등 기계 학습 파이프라인의 다른 단계에 필요한 도구들을 포함하고 있습니다. 이는 연구자와 개발자가 데이터를 더 효율적으로 처리하고, 모델의 성능을 정확히 평가할 수 있게 해줍니다.\n\nScikit-learn의 강점은 그의 범용성에 있습니다. 이는 다양한 산업과 연구 분야에서 사용될 수 있으며, 소규모 데이터셋부터 대규모 데이터셋까지 다양한 크기의 데이터를 처리할 수 있습니다. 또한, 오픈 소스 프로젝트로서, Scikit-learn은 지속적인 개선과 업데이트가 이루어지며, 사용자 커뮤니티로부터의 피드백과 기여를 받아 발전해 나갑니다.\n\n기계 학습 분야에 있어서, Scikit-learn은 특히 초보자에게 친화적인 학습 자원을 제공함으로써, 복잡한 이론과 알고리즘을 이해하는 데 중요한 역할을 합니다. 이러한 접근성과 범용성은 Scikit-learn을 기계 학습을 시작하는 이들에게 인기 있는 선택지로 만들었습니다.\n\nNLP\n\nNLP(자연어 처리)는 인간의 언어를 이해하고 해석하는 컴퓨터 알고리즘과 기술의 집합입니다. 이 분야는 컴퓨터 

비교를 위해 전체 문서에서 바로 검색해보기

In [9]:
# 자식 분할기를 생성합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300)

# DB를 생성합니다.
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=HuggingFaceBgeEmbeddings()
)

store = InMemoryStore()

# Retriever 를 생성합니다.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
# 문서를 검색기에 추가합니다. docs는 문서 목록이고, ids는 문서의 고유 식별자 목록입니다.
retriever.add_documents(docs, ids=None, add_to_docstore=True)

In [11]:
# 저장소의 모든 키를 리스트로 반환합니다.
list(store.yield_keys())

['77f96ba3-f106-4d63-9971-209255fdb6d3',
 '0cb97267-6f11-48cc-8085-23fb8c88aa4f']

In [12]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Word2Vec")

In [15]:
print(sub_docs[0].page_content)

Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.


In [16]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.get_relevant_documents("Word2Vec")

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [19]:
# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서의 일부를 출력합니다.
print(retrieved_docs[0].page_content[100:500])

문서의 길이: 7482


 2007년 David Cournapeau에 의해 프로젝트가 시작되었으며, 그 후로 커뮤니티의 광범위한 기여를 받아 현재까지 발전해왔습니다. Scikit-learn은 분류, 회귀, 군집화, 차원 축소 등 다양한 기계 학습 작업을 지원하며, 사용하기 쉬운 API와 함께 제공되어 연구자와 개발자가 복잡한 데이터 과학 문제를 해결할 수 있도록 돕습니다.

핵심 특징 중 하나는 다양한 기계 학습 모델을 일관된 인터페이스로 제공한다는 점입니다. 이는 사용자가 알고리즘 간에 쉽게 전환할 수 있게 하여, 최적의 모델을 찾는 과정을 단순화합니다. 또한, Scikit-learn은 사전 처리, 모델 선택, 평가 지표 등 기계 학습 파이프라인의 다른 단계에 필요한 도구들을 포함하고 있습니다. 이는 연구자와 개발자가 데이터를 더


이번엔 큰 문서를 부모-자식 청크로 나눠서 검색해보자

In [28]:
# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=900)
# 자식 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
# 부모보다 작은 문서를 생성해야 합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300)
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소입니다.
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=HuggingFaceBgeEmbeddings()
)
# 부모 문서의 저장 계층입니다.
store = InMemoryStore()

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
retriever = ParentDocumentRetriever(
    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,
    # 문서 저장소를 지정합니다.
    docstore=store,
    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

In [30]:
retriever.add_documents(docs)  # 문서를 retriever에 추가합니다.

In [31]:
len(list(store.yield_keys()))

20

In [39]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Word2Vec")
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(len(sub_docs))
print(sub_docs[0].page_content)
print(sub_docs[1].page_content)
print(sub_docs[2].page_content)
print(sub_docs[3].page_content)

4
Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.
Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.
Word2Vec

Word2Vec은 자연어 처리(NLP) 분야에서 널리 사용되는 획기적인 단어 임베딩 기법 중 하나입니다. 2013년 Google의 연구팀에 의해 개발되었으며, 단어를 벡터 공간에 매핑함으로써 컴퓨터가 단어의 의미를 수치적으로 이해할 수 있게 합니다. Word2Vec의 핵심 아이디어는 비슷한 맥락에서 사용되는 단어들은 비슷한 벡터를 가진다는 것입니다. 이를 통해 단어 간의 의미적 유사성과 관계를 벡터 연산을 통해 표현할 수 있습니다.
Word2Vec

Word2Vec은 자연어 처리(NLP) 분야에서 널리 사용되는 획기적인 단어 임베딩 기법 중 하나입니다. 2013년 Google의 연구팀에 의해 개발되었으며, 단어를 벡터 공간에 매핑함으로써 컴퓨터가 단어의 의미를 수치적으로 이해할 수 있게 합니다. Word2Vec의 핵심 아이디어는 비슷한 맥락에서 사용되는 단어들은 비슷한 벡터를 가진다는 것입니다. 이를 통해 단어 간의 의미적 유사성과 관계를 벡터 연산을 통해 표현할 수 있습니다.


In [36]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.get_relevant_documents("Word2Vec")

# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환합니다.
print(len(retrieved_docs))
print(retrieved_docs[0].page_content)
print(retrieved_docs[1].page_content)

2
Word2Vec의 성공 이후, 이와 유사한 다른 단어 임베딩 기법들도 개발되었습니다. 그러나 Word2Vec은 그 간결함과 효율성, 높은 성능으로 인해 여전히 광범위하게 사용되며, NLP 분야에서 기본적인 도구로 자리 잡았습니다. Word2Vec는 단순한 텍스트 데이터를 통해 복잡한 언어의 의미 구조를 학습할 수 있는 강력한 방법을 제공함으로써, 컴퓨터가 인간 언어를 이해하는 방식을 혁신적으로 개선하였습니다.
, SARIMA는 계절적 패턴을 고려해야 하는 경우에 사용됩니다. SARIMAX는 외부 요인을 모델에 포함시켜야 할 때 가장 적합한 선택입니다.

Word2Vec

Word2Vec은 자연어 처리(NLP) 분야에서 널리 사용되는 획기적인 단어 임베딩 기법 중 하나입니다. 2013년 Google의 연구팀에 의해 개발되었으며, 단어를 벡터 공간에 매핑함으로써 컴퓨터가 단어의 의미를 수치적으로 이해할 수 있게 합니다. Word2Vec의 핵심 아이디어는 비슷한 맥락에서 사용되는 단어들은 비슷한 벡터를 가진다는 것입니다. 이를 통해 단어 간의 의미적 유사성과 관계를 벡터 연산을 통해 표현할 수 있습니다.

Word2Vec은 크게 두 가지 모델 아키텍처로 구성됩니다: Continuous Bag-of-Words (CBOW)와 Skip-Gram입니다. CBOW 모델은 주변 단어(맥락)를 기반으로 특정 단어를 예측하는 반면, Skip-Gram 모델은 하나의 단어로부터 주변 단어들을 예측합니다. 두 모델 모두 딥러닝이 아닌, 단순화된 신경망 구조를 사용하여 대규모 텍스트 데이터에서 학습할 수 있으며, 매우 효율적입니다.

Word2Vec의 벡터 표현은 다양한 NLP 작업에 활용될 수 있습니다. 예를 들어, 단어의 유사도 측정, 문장이나 문서의 벡터 표현 생성, 기계 번역, 감정 분석 등이 있습니다. 또한, 벡터 연산을 통해 단어 간의 의미적 관계를 추론하는 것이 가능해집니다. 예를 들어, "king" - "man" + "woman"과 같은 벡터 연산을 수행하면, 결과적으